In [3]:
import json
import csv
import re
from datetime import datetime

# 상태 매핑 딕셔너리
STATUS_MAP = {
    '기안': 'DRAFT',
    '승인': 'APPROVAL',
    '합의': 'AGREEMENT'
}

def extract_document_id(source_id):
    """sourceId에서 document_id 추출 (예: doc_2985802_02 → 2985802)"""
    match = re.search(r'doc_(\d+)_', source_id)
    return match.group(1) if match else None

def parse_approval_date(date_str):
    """날짜 문자열을 유닉스 타임스탬프 밀리초로 변환"""
    dt = datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')
    return int(dt.timestamp() * 1000)

def process_cmds(csv_file, input_cmds, output_cmds):
    # CSV 파일 읽기
    csv_data = {}
    with open(csv_file, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            doc_id = row['document_id']
            if doc_id not in csv_data:
                csv_data[doc_id] = []
            csv_data[doc_id].append(row)
    
    # 불일치 리스트
    mismatch_list = []
    line_count = 0
    error_count = 0
    
    # CMDS 파일 처리
    with open(input_cmds, 'r', encoding='utf-8') as f_in, \
         open(output_cmds, 'w', encoding='utf-8') as f_out:
        
        for line in f_in:
            line = line.strip()
            if not line:
                continue
            
            line_count += 1
            
            # 진행상황 표시 (1000건마다)
            if line_count % 1000 == 0:
                print(f'처리 중... {line_count}건')
            
            # "addDocument " 접두어 제거
            if line.startswith('addDocument '):
                json_str = line[12:]  # "addDocument " 는 12글자
            else:
                json_str = line
            
            # JSON 파싱 (에러 처리 추가)
            try:
                data = json.loads(json_str)
            except json.JSONDecodeError as e:
                error_count += 1
                print(f'\n⚠️ JSON 파싱 에러 (줄 {line_count}): {str(e)}')
                print(f'   문제 줄 미리보기: {line[:100]}...')
                # 파싱 실패한 줄은 원본 그대로 출력
                f_out.write(line + '\n')
                continue
            
            # sourceId에서 document_id 추출
            source_id = data.get('sourceId', '')
            doc_id = extract_document_id(source_id)
            
            if doc_id and doc_id in csv_data:
                csv_rows = csv_data[doc_id]
                
                # 제목 비교 (CMDS의 title vs CSV의 post_title)
                cmds_title = data.get('title', '')
                csv_title = csv_rows[0]['post_title']
                
                if cmds_title != csv_title:
                    # 제목 불일치 - 리스트에 추가하고 원본 그대로 출력
                    mismatch_list.append({
                        'doc_id': doc_id,
                        'cmds_title': cmds_title,
                        'csv_title': csv_title
                    })
                    output_line = 'addDocument ' + json.dumps(data, ensure_ascii=False, separators=(',', ':'))
                    f_out.write(output_line + '\n')
                    continue
                
                # 제목 일치 - activities 업데이트
                for activity in data.get('activities', []):
                    activity_type = activity.get('type', '')
                    activity_name = activity.get('name', '')
                    
                    # CSV에서 매칭되는 행 찾기
                    for csv_row in csv_rows:
                        csv_status = STATUS_MAP.get(csv_row['status'], '')
                        csv_approver = csv_row['approver']
                        
                        # type과 name이 모두 일치하는지 확인
                        if (activity_type == csv_status and 
                            activity_name in csv_approver):
                            
                            # actionDate 업데이트
                            approval_date = csv_row['approval_date']
                            activity['actionDate'] = parse_approval_date(approval_date)
                            break
            
            # JSON을 한 줄로 변환하여 출력 (addDocument 접두어 포함)
            output_line = 'addDocument ' + json.dumps(data, ensure_ascii=False, separators=(',', ':'))
            f_out.write(output_line + '\n')
    
    # 처리 완료 후 로그 출력
    print(f'\n총 처리: {line_count}건')
    if error_count > 0:
        print(f'⚠️ JSON 파싱 에러: {error_count}건')
    
    if len(mismatch_list) > 0:
        # 불일치가 있으면 항상 파일로 저장
        with open('title_mismatch.log', 'w', encoding='utf-8') as log:
            for item in mismatch_list:
                log.write(f"Doc ID: {item['doc_id']}\n")
                log.write(f"CMDS title: {item['cmds_title']}\n")
                log.write(f"CSV post_title: {item['csv_title']}\n\n")
        print(f'⚠️ 제목 불일치: {len(mismatch_list)}건 (title_mismatch.log 파일 확인)')
    else:
        print('✓ 모든 제목 일치')

# 실행
if __name__ == '__main__':
    csv_file = r"C:\Users\LEEJUHWAN\Downloads\crawling_approval_data_2010.csv"  # CSV 파일 경로
    input_cmds = r"C:\Users\LEEJUHWAN\Downloads\documents_2010_imgFixed.cmds"  # 입력 CMDS 파일
    output_cmds = r"C:\Users\LEEJUHWAN\Downloads\documents_2010_imgFixed_timefixed.cmds"  # 출력 CMDS 파일
    
    process_cmds(csv_file, input_cmds, output_cmds)
    print('처리 완료!')


총 처리: 840건
✓ 모든 제목 일치
처리 완료!


In [1]:
# id 같은데 제목 다를경우 콘솔에 찍히게 수정

import json
import csv
import re
from datetime import datetime

# 상태 매핑 딕셔너리
STATUS_MAP = {
    '기안': 'DRAFT',
    '승인': 'APPROVAL',
    '합의': 'AGREEMENT'
}

def extract_document_id(source_id):
    """sourceId에서 document_id 추출 (예: doc_2985802_02 → 2985802)"""
    match = re.search(r'doc_(\d+)_', source_id)
    return match.group(1) if match else None

def parse_approval_date(date_str):
    """날짜 문자열을 유닉스 타임스탬프 밀리초로 변환"""
    dt = datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')
    return int(dt.timestamp() * 1000)

def process_cmds(csv_file, input_cmds, output_cmds):
    # CSV 파일 읽기
    csv_data = {}
    with open(csv_file, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            doc_id = row['document_id']
            if doc_id not in csv_data:
                csv_data[doc_id] = []
            csv_data[doc_id].append(row)
    
    # 불일치 리스트
    mismatch_list = []
    line_count = 0
    
    # CMDS 파일 처리
    with open(input_cmds, 'r', encoding='utf-8') as f_in, \
         open(output_cmds, 'w', encoding='utf-8') as f_out:
        
        for line in f_in:
            line = line.strip()
            if not line:
                continue
            
            line_count += 1
            
            # 진행상황 표시 (1000건마다)
            if line_count % 1000 == 0:
                print(f'처리 중... {line_count}건')
            
            # JSON 파싱
            data = json.loads(line)
            
            # sourceId에서 document_id 추출
            source_id = data.get('sourceId', '')
            doc_id = extract_document_id(source_id)
            
            if doc_id and doc_id in csv_data:
                csv_rows = csv_data[doc_id]
                
                # 제목 비교 (CMDS의 title vs CSV의 post_title)
                cmds_title = data.get('title', '')
                csv_title = csv_rows[0]['post_title']
                
                if cmds_title != csv_title:
                    # 제목 불일치 - 리스트에 추가하고 원본 그대로 출력
                    mismatch_list.append({
                        'doc_id': doc_id,
                        'cmds_title': cmds_title,
                        'csv_title': csv_title
                    })
                    output_line = json.dumps(data, ensure_ascii=False, separators=(',', ':'))
                    f_out.write(output_line + '\n')
                    continue
                
                # 제목 일치 - activities 업데이트
                for activity in data.get('activities', []):
                    activity_type = activity.get('type', '')
                    activity_name = activity.get('name', '')
                    
                    # CSV에서 매칭되는 행 찾기
                    for csv_row in csv_rows:
                        csv_status = STATUS_MAP.get(csv_row['status'], '')
                        csv_approver = csv_row['approver']
                        
                        # type과 name이 모두 일치하는지 확인
                        if (activity_type == csv_status and 
                            activity_name in csv_approver):
                            
                            # actionDate 업데이트
                            approval_date = csv_row['approval_date']
                            activity['actionDate'] = parse_approval_date(approval_date)
                            break
            
            # JSON을 한 줄로 변환하여 출력
            output_line = json.dumps(data, ensure_ascii=False, separators=(',', ':'))
            f_out.write(output_line + '\n')
    
    # 처리 완료 후 불일치 로그 출력
    print(f'\n총 처리: {line_count}건')
    
    if len(mismatch_list) > 0:
        # 불일치가 있으면 항상 파일로 저장
        with open('title_mismatch.log', 'w', encoding='utf-8') as log:
            for item in mismatch_list:
                log.write(f"Doc ID: {item['doc_id']}\n")
                log.write(f"CMDS title: {item['cmds_title']}\n")
                log.write(f"CSV post_title: {item['csv_title']}\n\n")
        print(f'⚠️ 제목 불일치: {len(mismatch_list)}건 (title_mismatch.log 파일 확인)')
    else:
        print('✓ 모든 제목 일치')

# 실행
if __name__ == '__main__':
    csv_file = r"C:\Users\LEEJUHWAN\Downloads\crawling_approval_data_2010.csv"  # CSV 파일 경로
    input_cmds = r"C:\Users\LEEJUHWAN\Downloads\documents_2010_imgFixed.cmds"  # 입력 CMDS 파일
    output_cmds = r"C:\Users\LEEJUHWAN\Downloads\documents_2010_imgFixed_timefixed.cmds"  # 출력 CMDS 파일
    
    process_cmds(csv_file, input_cmds, output_cmds)
    print('처리 완료!')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
import json
import csv
import re
from datetime import datetime

# 상태 매핑 딕셔너리
STATUS_MAP = {
    '기안': 'DRAFT',
    '승인': 'APPROVAL',
    '합의': 'AGREEMENT'
}

def extract_document_id(source_id):
    """sourceId에서 document_id 추출 (예: doc_2985802_02 → 2985802)"""
    match = re.search(r'doc_(\d+)_', source_id)
    return match.group(1) if match else None

def parse_approval_date(date_str):
    """날짜 문자열을 유닉스 타임스탬프 밀리초로 변환"""
    dt = datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')
    return int(dt.timestamp() * 1000)

def process_cmds(csv_file, input_cmds, output_cmds):
    # CSV 파일 읽기
    csv_data = {}
    with open(csv_file, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            doc_id = row['document_id']
            if doc_id not in csv_data:
                csv_data[doc_id] = []
            csv_data[doc_id].append(row)
    
    # CMDS 파일 처리
    with open(input_cmds, 'r', encoding='utf-8') as f_in, \
         open(output_cmds, 'w', encoding='utf-8') as f_out:
        
        for line in f_in:
            line = line.strip()
            if not line:
                continue
            
            # JSON 파싱
            data = json.loads(line)
            
            # sourceId에서 document_id 추출
            source_id = data.get('sourceId', '')
            doc_id = extract_document_id(source_id)
            
            if doc_id and doc_id in csv_data:
                csv_rows = csv_data[doc_id]
                
                # activities 업데이트
                for activity in data.get('activities', []):
                    activity_type = activity.get('type', '')
                    activity_name = activity.get('name', '')
                    
                    # CSV에서 매칭되는 행 찾기
                    for csv_row in csv_rows:
                        csv_status = STATUS_MAP.get(csv_row['status'], '')
                        csv_approver = csv_row['approver']
                        
                        # type과 name이 모두 일치하는지 확인
                        if (activity_type == csv_status and 
                            activity_name in csv_approver):
                            
                            # actionDate 업데이트
                            approval_date = csv_row['approval_date']
                            activity['actionDate'] = parse_approval_date(approval_date)
                            break
            
            # JSON을 한 줄로 변환하여 출력
            output_line = json.dumps(data, ensure_ascii=False, separators=(',', ':'))
            f_out.write(output_line + '\n')

# 실행
if __name__ == '__main__':
    csv_file = 'approval_data.csv'  # CSV 파일 경로
    input_cmds = 'input.cmds'  # 입력 CMDS 파일
    output_cmds = 'output.cmds'  # 출력 CMDS 파일
    
    process_cmds(csv_file, input_cmds, output_cmds)
    print('처리 완료!')